In [8]:
from matplotlib import pyplot as plt
import matplotlib as mpl
plt.style.use("ggplot")
mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 14
from scipy.integrate._ivp.ivp import OdeResult
from scipy.optimize import curve_fit
import seaborn as sns
import numpy as np
import tqdm, os
import itertools

from ars_247_2025_final_project.tram_model import TRAM_Model
from ars_247_2025_final_project.tools import gaussian_pulse, double_sigmoid_pulse

In [4]:
OUTPUT_DIR = os.path.abspath(os.path.join("..", "Final_Project_PNG"))
print(f"Saving files to {OUTPUT_DIR}...")
HOUR = 3600
MINUTE = 60
# transcriptional pulse
transcriptional_pulse = {
    'peak_time': 1.5 * HOUR,
    'peak_value': 1,
    'width': 0.5 * HOUR
}

# phosphorylation pulse
phosphorylation_pulse = {
    'peak_time': 1.5 * HOUR // 4,
    'peak_value': 1,
    'width': 0.5 * HOUR // 4 # 45 mins peak phosphorylation pulse
}

# optogenetic pulse
optogenetic_pulse = {
    'peak_time': 1.5 * HOUR // 10, # peak at 9 mins
    'peak_value': 1,
    'width': 0.5 * HOUR // 10 # 3 mins peak activity of pulse
}

Saving files to /Users/alexandresathler/Documents/BioE_Program/Bioengineering Courses/ARS_247_2025_Final_Project/Final_Project_PNG...


## Figure 2

In [18]:
system_params = {
    'PROM': 1,
    'V_A': 0.47, #um/s - mean for dynein / kinesin
    'R_M': 2.5, #um - mean for mammalian cells is 2.5; best results w/ 20
    'K_TEV': 0.015, #/s - physiological is ~0.15
    'K_TVMV': 0.015, #/s - physiological is ~0.15
    'K_TRAM_DEG': 0.00001, #/s - got interesting results with this being 0.01; physiological is ~0.00001
    'K_TF_DEG': 0.01 #/s - got interesting results with this being 0.1; physiological is ~0.01 w/ fast degron
}

In [ ]:
# Creating the linear parameter space for each parameter of interest
linear_parameter_space = np.concat([1 / np.arange(2, 8, 2)[::-1], np.ndarray([1]), np.arange(2, 8, 2)])
speed_ranges = linear_parameter_space * system_params['V_A']
protease_ranges = linear_parameter_space * system_params['K_TEV']
deg_tram_ranges = linear_parameter_space * system_params['K_TRAM_DEG']
deg_tf_ranges = linear_parameter_space * system_params['K_TF_DEG']

# Combinatorially constructing all possible parameter combinations based on linear parameter space
all_combinations = list(itertools.product(*[speed_ranges, protease_ranges, deg_tram_ranges, deg_tf_ranges]))
models_comb = []
solns_comb = []
for speedx, protx, degrx, degtx in tqdm.tqdm(all_combinations):
    system_params['V_A'] = speedx
    system_params['K_TEV'], system_params['K_TVMV'] = protx, protx
    system_params['K_TRAM_DEG'], system_params['K_TF_DEG'] = degrx, degtx

    this_model = TRAM_Model(
        n_domains=6,
        pulse='gaussian',
        pulse_params='gaussian_pulse',
        sys_config=system_params
    )





[(np.float64(0.0024999999999999996), np.float64(1.6666666666666667e-06), np.float64(0.0016666666666666666)), (np.float64(0.0024999999999999996), np.float64(1.6666666666666667e-06), np.float64(0.0025)), (np.float64(0.0024999999999999996), np.float64(1.6666666666666667e-06), np.float64(0.005)), (np.float64(0.0024999999999999996), np.float64(1.6666666666666667e-06), np.float64(0.01)), (np.float64(0.0024999999999999996), np.float64(1.6666666666666667e-06), np.float64(0.02)), (np.float64(0.0024999999999999996), np.float64(1.6666666666666667e-06), np.float64(0.04)), (np.float64(0.0024999999999999996), np.float64(1.6666666666666667e-06), np.float64(0.06)), (np.float64(0.0024999999999999996), np.float64(2.5e-06), np.float64(0.0016666666666666666)), (np.float64(0.0024999999999999996), np.float64(2.5e-06), np.float64(0.0025)), (np.float64(0.0024999999999999996), np.float64(2.5e-06), np.float64(0.005)), (np.float64(0.0024999999999999996), np.float64(2.5e-06), np.float64(0.01)), (np.float64(0.0024

### Figure 2a